## Librerías

In [1]:
#Instalar librerías
!pip install pymongo
!pip install emoji==1.6.3
!pip install tensorflow

In [2]:
#Importar librerías
import pymongo
import time
import emoji
import re
import tensorflow as tf
from tensorflow.keras.optimizers import Adam # type: ignore

## Datos de conexión

In [3]:
MONGO_HOST = "localhost"
MONGO_PUERTO = "27017"
MONGO_URI = "mongodb://" + MONGO_HOST + ":" + MONGO_PUERTO + "/"
MONGO_BASEDATOS = "clasificador_sentimientos"

try:
    cliente = pymongo.MongoClient(MONGO_URI)
    base_datos = cliente[MONGO_BASEDATOS]
    coleccion_general = base_datos["publicaciones"]
    coleccion_bots_entrenamiento = base_datos['bots_entrenamiento']
    coleccion_humanos_entrenamiento = base_datos['humanos_entrenamiento']
    coleccion_datos_entrenamiento = base_datos["datos_entrenamiento"]
    coleccion_bots = base_datos["bots"]
    coleccion_humanos = base_datos["humanos"]
except pymongo.errors.InvalidURI: # type: ignore
    print("La url de conexión es incorrecta")

## Funciones

In [4]:
def obtener_longitud_arreglo(campo, subcampo):
    if campo in documento:
        if subcampo in documento[campo]:
            elemento = documento[campo][subcampo]
            cantidad_elemento = len(elemento)
        else: 
            cantidad_elemento = 0
    else: 
        cantidad_elemento = 0

    return cantidad_elemento

In [5]:
def crear_campo_base_datos(coleccion, documento, nombre_campo, valor):
    if nombre_campo in documento:
        documento[nombre_campo] = valor
    else:
        filtro = {"_id": documento["_id"]} 
        actualizacion = {"$set": {"para_los_bots." + nombre_campo: valor}}
        coleccion.update_one(filtro, actualizacion)

In [6]:
def limpiar_texto():
    global texto_limpio
    texto_limpio = texto_limpio.replace("\n", "")
    limpiar_objeto(cantidad_urls, "urls", "url")
    #limpiar_objeto(cantidad_menciones, "mentions", "username",  "@")
    #limpiar_objeto(cantidad_hashtags, "hashtags", "tag", "#")

    texto_limpio = texto_limpio.replace('‼️', '‼').strip()
    emojis_unicode = emoji.get_emoji_regexp()
    emojis_texto = []
    for c in emojis_unicode.findall(texto_limpio):
         if c != '‼':
            emojis_texto.append(c)
            texto_limpio = texto_limpio.replace(c, "")
    crear_campo_base_datos(coleccion_general, documento,"emojis", emojis_texto)

    texto_limpio = texto_limpio.replace("  ", " ")

In [7]:
def limpiar_objeto(cantidad_objeto, campo, subcampo, opcional=""):
     global texto_limpio
     for i in range(cantidad_objeto):
        objeto = documento["entities"][campo][i][subcampo]
        texto_limpio = texto_limpio.replace(opcional + objeto, "")

In [8]:
def obtenerDatosEntrenamiento():
    for documento in coleccion_bots_entrenamiento.find().limit(cantidad_bots_entrenamiento):
        datos_entrenamiento.append(documento)
    
    for documento in coleccion_humanos_entrenamiento.find().limit(cantidad_humanos_entrenamiento):
        datos_entrenamiento.append(documento)

    for documento in datos_entrenamiento:
        entrada = []
        entrada.append(documento["para_los_bots"]["numero_urls"])
        entrada.append(documento["para_los_bots"]["numero_menciones"])
        entrada.append(documento["para_los_bots"]["numero_hashtags"])
        entrada.append(documento["public_metrics"]["retweet_count"])
        entrada.append(documento["public_metrics"]["like_count"])
        entrada.append(documento["public_metrics"]["quote_count"])
        entrada.append(documento["public_metrics"]["impression_count"])

        entradas_entrenamiento.append(entrada)
        salidas_entrenamiento.append(documento["para_los_bots"]["bot"])

In [9]:
def obtenerDatosPruebas():
    for documento in coleccion_bots_entrenamiento.find().skip(cantidad_bots_entrenamiento):
        datosPruebas.append(documento)
    
    for documento in coleccion_humanos_entrenamiento.find().skip(cantidad_humanos_entrenamiento):
        datosPruebas.append(documento)

    for documento in datosPruebas:
        entrada = []
        entrada.append(documento["para_los_bots"]["numero_urls"])
        entrada.append(documento["para_los_bots"]["numero_menciones"])
        entrada.append(documento["para_los_bots"]["numero_hashtags"])
        entrada.append(documento["public_metrics"]["retweet_count"])
        entrada.append(documento["public_metrics"]["like_count"])
        entrada.append(documento["public_metrics"]["quote_count"])
        entrada.append(documento["public_metrics"]["impression_count"])

        entradas_pruebas.append(entrada)
        salidas_pruebas.append(documento["para_los_bots"]["bot"])

## Limpieza de las publicaciones

In [10]:
#Limpiar Retuits y publicaciones que no estén en español
tiempo_inicial = time.time() #Iniciar cronometro

for documento in coleccion_general.find():
    if (documento["text"].count("RT") >= 1) or (documento["lang"] != "es"):
        coleccion_general.delete_one({'_id': documento["_id"]})

tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para limpiar los retuits y las publicaciones que no están en español: {round(tiempo_transcurrido, 3)} segundos")

Tiempo transcurrido para limpiar los retuits y las publicaciones que no están en español: 115.808 segundos


In [11]:
# Eliminar publicaciones repetidas
tiempo_inicial = time.time() #Iniciar cronometro

for documento in coleccion_general.find():
    duplicados = coleccion_general.find({'id': documento["id"]})
    duplicados = [dup for dup in duplicados if dup["_id"] != documento["_id"]]
    for duplicado in duplicados:
        coleccion_general.delete_one({'_id': duplicado["_id"]})
        
tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para limpiar las publicaciones duplicadas: {round(tiempo_transcurrido, 3)} segundos")

Tiempo transcurrido para limpiar las publicaciones duplicadas: 1767.324 segundos


## Obtener los datos importantes para los clasificadores

In [12]:
tiempo_inicial = time.time() #Iniciar cronometro
for documento in coleccion_general.find():

    cantidad_urls = obtener_longitud_arreglo("entities", "urls")
    crear_campo_base_datos(coleccion_general, documento, "numero_urls", cantidad_urls)

    cantidad_menciones = obtener_longitud_arreglo("entities", "mentions")
    crear_campo_base_datos(coleccion_general, documento, "numero_menciones", cantidad_menciones)

    cantidad_hashtags = obtener_longitud_arreglo("entities", "hashtags")
    crear_campo_base_datos(coleccion_general, documento, "numero_hashtags", cantidad_hashtags)

    texto = documento["text"]
    texto_limpio = texto
    limpiar_texto()
    crear_campo_base_datos(coleccion_general, documento, "texto_limpio", texto_limpio)

    caracteres_a_buscar = r"[¿?¡!]"
    signos_interrogacion_exclamacion = len(re.findall(caracteres_a_buscar, texto_limpio))
    crear_campo_base_datos(coleccion_general, documento, "numero_signos_interrogacion_exclamacion", signos_interrogacion_exclamacion)

    caracteres_a_buscar = r"[^a-zA-Z0-9\s¿?¡!‼().…,:;“”áéíóúÁÉÍÓÚüÜñÑ\"'-]"
    simbolos_especiales = len(re.findall(caracteres_a_buscar, texto_limpio))
    crear_campo_base_datos(coleccion_general, documento, "numero_simbolos_especiales", simbolos_especiales)

tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para obtener los datos importantes: {round(tiempo_transcurrido, 3)} segundos")

Tiempo transcurrido para obtener los datos importantes: 230.081 segundos


## Clasificación automatizada

In [13]:
tiempo_inicial = time.time() #Iniciar cronometro

for documento in coleccion_general.find():
    retuits = documento["public_metrics"]["retweet_count"]
    comentarios =documento["public_metrics"]["reply_count"]
    likes = documento["public_metrics"]["like_count"]
    quotes =documento["public_metrics"]["quote_count"]
    vistas = documento["public_metrics"]["impression_count"]
    
    suma = retuits + likes + comentarios + quotes
    # 1. Si no tiene nada en el texto que no sean menciones, hashtags o urls es bot
    # 2. Si la cantidad de vistas es menor a cualquiera de los otros campos (Sin contar los hashtags y menciones) es bot.
    # 3. Si la suma de los retuits, likes, comentarios (reply) y quote_count es menor al 7% de las vistas, es bot
    if(documento["para_los_bots"]["texto_limpio"].replace(" ", "") == "" or
       max(retuits, comentarios, likes, quotes, vistas) != vistas or
       suma < (vistas * .05)):
        crear_campo_base_datos(coleccion_general, documento, "bot", 1)
        coleccion_bots_entrenamiento.insert_one(documento)
    else:
        crear_campo_base_datos(coleccion_general, documento, "bot", 0)
        coleccion_humanos_entrenamiento.insert_one(documento)

for humano in coleccion_humanos_entrenamiento.find():
    crear_campo_base_datos(coleccion_humanos_entrenamiento, humano, "bot", 0)

for bot in coleccion_bots_entrenamiento.find():
    crear_campo_base_datos(coleccion_bots_entrenamiento, bot, "bot", 1)
    
tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para separar las publicaciones: {round(tiempo_transcurrido, 3)} segundos")

Tiempo transcurrido para separar las publicaciones: 73.777 segundos


## Combinar datos de entrenamiento

In [14]:
tiempo_inicial = time.time() #Iniciar cronometro
publicaciones = []

for documento in coleccion_bots_entrenamiento.find():
    coleccion_datos_entrenamiento.insert_one(documento)

for documento in coleccion_humanos_entrenamiento.find():
    coleccion_datos_entrenamiento.insert_one(documento)

tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para combinar los datos de entrenamiento: {round(tiempo_transcurrido, 3)} segundos")

Tiempo transcurrido para combinar los datos de entrenamiento: 24.553 segundos


## Perceptrón multicapa

### Obtener datos de prueba y entrenamiento

In [15]:
cantidad_bots_entrenamiento = 4875
cantidad_humanos_entrenamiento = 4875
entradas_entrenamiento = []
salidas_entrenamiento = []
entradas_pruebas = []
salidas_pruebas = []
datos_entrenamiento = []
datosPruebas = []

obtenerDatosEntrenamiento()
obtenerDatosPruebas()

### Creación del modelo

In [16]:
neuronas_capaEntrada = 7
neuronas_capaOculta1 = 22 #El más alto ha sido 22 con un 85% de precisión
neuronas_capaSalida = 1

#Creando el perceptron
perceptron_multicapa = tf.keras.Sequential([
    # relu, tanh y sigmoid suelen utilizarse para las capas ocultas
    tf.keras.layers.Dense(neuronas_capaOculta1, activation='tanh', input_shape=(neuronas_capaEntrada,)), #Capa de entrada y capa oculta
    tf.keras.layers.Dense(neuronas_capaSalida, activation='sigmoid') #Capa de salida
])

# Compilando el perceptron
perceptron_multicapa.compile(optimizer=Adam(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

### Entrenamiento del modelo

In [17]:
epochs=200 #El más alto ha sido 200
batch_size=128

# Entrenando el perceptron
tiempo_inicial = time.time() #Iniciar cronometro
perceptron_multicapa.fit(entradas_entrenamiento, salidas_entrenamiento, epochs=epochs, batch_size=batch_size)
tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para entrenar el modelo: {round(tiempo_transcurrido, 3)} segundos")

print("Precición del perceptron:")
perceptron_multicapa.evaluate(entradas_pruebas, salidas_pruebas)

Epoch 1/200
77/77 [==============================] - 3s 2ms/step - loss: 0.2827 - accuracy: 0.8802
Epoch 2/200
77/77 [==============================] - 0s 2ms/step - loss: 0.1744 - accuracy: 0.9434
Epoch 3/200
77/77 [==============================] - 0s 2ms/step - loss: 0.2176 - accuracy: 0.9231
Epoch 4/200
77/77 [==============================] - 0s 2ms/step - loss: 0.2125 - accuracy: 0.9443
Epoch 5/200
77/77 [==============================] - 0s 2ms/step - loss: 0.2399 - accuracy: 0.9152
Epoch 6/200
77/77 [==============================] - 0s 3ms/step - loss: 0.2123 - accuracy: 0.9269
Epoch 7/200
77/77 [==============================] - 0s 2ms/step - loss: 0.1889 - accuracy: 0.9361
Epoch 8/200
77/77 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9469
Epoch 9/200
77/77 [==============================] - 0s 2ms/step - loss: 0.1749 - accuracy: 0.9462
Epoch 10/200
77/77 [==============================] - 0s 2ms/step - loss: 0.1422 - accuracy: 0.9558
Epoch 11/

[0.16570089757442474, 0.909590482711792]

### Clasificación de todas las publicaciones

In [18]:
tiempo_inicial = time.time() #Iniciar cronometro
for publicacion in coleccion_general.find():
    datos_entrada_clasificador = []
    datos_entrada_clasificador.append(publicacion["para_los_bots"]["numero_urls"])
    datos_entrada_clasificador.append(publicacion["para_los_bots"]["numero_menciones"])
    datos_entrada_clasificador.append(publicacion["para_los_bots"]["numero_hashtags"])
    datos_entrada_clasificador.append(publicacion["public_metrics"]["retweet_count"])
    datos_entrada_clasificador.append(publicacion["public_metrics"]["like_count"])
    datos_entrada_clasificador.append(publicacion["public_metrics"]["quote_count"])
    datos_entrada_clasificador.append(publicacion["public_metrics"]["impression_count"])

    salida_clasificador = perceptron_multicapa.predict([datos_entrada_clasificador])
    salida_clasificador = [1 if salida_clasificador > 0.5 else 0 in salida_clasificador]

    if salida_clasificador[0] == 1:
        try:
            coleccion_bots.insert_one(publicacion)
        except:
            print("Ya existe este tuits en la base de datos de humanos")
    else:
        try:
            coleccion_humanos.insert_one(publicacion)
        except:
            print("Ya existe este tuits en la base de datos de bots")

tiempo_transcurrido = time.time() - tiempo_inicial
print(f"Tiempo transcurrido para para clasificar las 40k publicaciones: {round(tiempo_transcurrido, 3)} segundos")
cliente.close()

1/1 [==============================] - 0s 51ms/step
Tiempo transcurrido para para clasificar las 40k publicaciones: 4268.015 segundos
